In [13]:
import requests
import re

url = 'https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieDtl.do'

# post 요청의 본문에 해당하는 데이터
payload = {
    'code': '20247450',
    'sType': '', # 검색 타입을 지정하는 필드 
    'titleYN': 'Y', # 영화 제목 포함 여부 
    'etcParam': '',
    'isOuterReq': 'false',
    # 'CSRFToken': 'j62iniJVHjKeqJIqvWlnt9lZI10iQDtDd_uVBMUH72M', # csrf 공격을 방지하기 위한 토큰 
}

# 추가적인 메타데이터를 포함
# 클라이언트의 정보를 서버에 전달, 서버가 요청을 올바르게 처리하도록 한다 
# 클라이언트의 유효성 검증을 위해 사용 
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Referer': 'https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do', # 요청이 발생한 출처를 서버에 알린다 
    # 'Cookie': 'JSESSIONID=6TbBnpld1ngM47CxslWzTzJgC4QmVy6cRVgMYLwvycLLBjLrWJJh!1013290353!-1702303710',  # 세션 쿠키
    'X-Requested-With': 'XMLHttpRequest', # ajax 요청임을 명시 
}

response = requests.post(url, headers=headers, data=payload)
# 빈 줄 제거 (줄 사이 공백 제거)
cleaned_text = "\n".join(line for line in response.text.splitlines() if line.strip())
print(cleaned_text)

<!--[if IE]>
<script src="/kobis/web/comm/commjs/html5shiv.js"></script>
<![endif]-->
<!-- 기본 css -->
<link rel="stylesheet" type="text/css" href="/kobis/web/comm/commcss/default.css">
<link rel="stylesheet" type="text/css" href="/kobis/web/comm/commcss/jquery-ui-1.12.1.min.css" />
<link rel="stylesheet" type="text/css" href="/kobis/web/comm/commcss/common.css" />
<script type="text/javascript" src="/kobis/web/comm/commjs/jquery-1.12.4.js"></script>
<script type="text/javascript" src="/kobis/web/comm/commjs/jquery.dotdotdot.min.js"></script>
<script type="text/javascript" src="/kobis/web/comm/commjs/jquery.bxslider.js"></script>
<script type="text/javascript" src="/kobis/web/comm/commjs/comm_script.js"></script>
<script type="text/javascript" src="/kobis/web/comm/commjs/jquery-ui-1.13.2.min.js"></script>
<script type="text/javascript" src="/kobis/web/comm/commjs/jquery.ui.datepicker-ko.js"></script>
<script type="text/javascript" src="/kobis/web/comm/commjs/kobis.js"></script>
<link re

In [14]:
from bs4 import BeautifulSoup

# BeautifulSoup 파싱
soup = BeautifulSoup(cleaned_text, 'html.parser')

# 제목 검색
title = soup.find('strong', class_='tit').text  # "검은 수녀들" 텍스트를 추출
print(title)

검은 수녀들


In [15]:
# ajax 데이터를 가져올 때마다 달라지는 메타데이터
# code
# csrf

In [30]:
# 첫 번째 페이지에서 데이터 찾기

import requests
from bs4 import BeautifulSoup

# URL
url = "https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do#none"

# 요청 보내기
response = requests.get(url)

# HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# span 클래스가 'ellip'인 태그 안에 있는 모든 <a> 태그 중 onclick 속성 있는 항목 찾기
movie_links = soup.find_all('span', class_='ellip')

# 원하는 영화 제목 찾기
search_title = "탈주"
for span in movie_links:
    a_tag = span.find('a', onclick=True)
    if a_tag and a_tag.text.strip() == search_title:
        print(f"영화 제목 '{search_title}' 찾음!")
        break
else:
    print(f"영화 제목 '{search_title}'을 찾을 수 없습니다.")
    
    
# 영화 제목을 찾긴 했다.. 그런데 이 코드가 가능한건 첫 번째 페이지 한정
# 목록 데이터를 불러오는게 페이징이 아니라 ajax 방식이라 전체에서 특정 데이터를 못 찾는다 


영화 제목 '탈주' 찾음!


In [88]:
import requests
from bs4 import BeautifulSoup
import re

# URL
base_url = "https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do"

# 공백 및 특수문자 제거 함수
def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9가-힣]', '', text)

# 영화 제목 검색
search_title = input("검색할 영화 제목을 입력하세요: ")

# 결과 저장용 리스트
found_movies = []

# 첫 페이지 요청으로 CSRFToken 추출
session = requests.Session()
initial_response = session.get(base_url)
initial_soup = BeautifulSoup(initial_response.text, 'html.parser')
csrf_token = initial_soup.find('input', {'name': 'CSRFToken'})['value']

# 페이지 순회
for page in range(1, 2000):
    payload = {
        'CSRFToken': csrf_token,
        'curPage': page,
        'sPrdtStatStr': '개봉',
        'sMovTypeStr': '장편'
    }

    # POST 요청
    response = session.post(base_url, data=payload)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 영화 제목 검색
    movie_links = soup.find_all('span', class_='ellip')
    for span in movie_links:
        a_tags = span.find_all('a', onclick=True)
        for a_tag in a_tags:
            movie_text = a_tag.text.strip()
            cleaned_search_title = clean_text(search_title)
            cleaned_movie_text = clean_text(movie_text)

            # 비교 조건 확인
            if cleaned_search_title in cleaned_movie_text or cleaned_movie_text in cleaned_search_title:
                found_movies.append((page, movie_text))
                print(f"페이지 {page}: 영화 제목 '{movie_text}'을(를) 찾았습니다!")

    # 페이지 내용 확인
    if not movie_links:
        print(f"페이지 {page}: 영화 데이터를 찾을 수 없습니다. 종료합니다.")
        break

# 최종 결과 출력
if found_movies:
    print("\n찾은 영화 목록:")
    for page, title in found_movies:
        print(f"페이지 {page}: {title}")
else:
    print(f"'{search_title}'에 해당하는 영화를 찾을 수 없습니다.")
    
# 순차적으로 찾긴 찾는데 페이지 숫자를 바꾸며 진행하니 조회 속도가 너무 느리다..


페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 1: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페이지 2: 영화 제목 ''을(를) 찾았습니다!
페

KeyboardInterrupt: 

In [104]:
import requests
from bs4 import BeautifulSoup
import re
from concurrent.futures import ThreadPoolExecutor

# URL 및 세션 설정
base_url = "https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do"

# 공백 및 특수문자 제거 함수
def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9가-힣]', '', text)

# 데이터 요청 함수
def fetch_page(page, csrf_token, search_title):
    payload = {
        'CSRFToken': csrf_token,
        'curPage': page,
        'sPrdtStatStr': '개봉',
        'sMovTypeStr': '장편'
    }
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.post(base_url, headers=headers, data=payload)
    soup = BeautifulSoup(response.text, 'html.parser')
    movie_links = soup.find_all('span', class_='ellip')

    results = []
    for span in movie_links:
        a_tags = span.find_all('a', onclick=True)
        for a_tag in a_tags:
            movie_text = a_tag.text.strip()
            # 디버깅: 비교 대상 출력
            # print(f"디버깅: 비교 -> '{clean_text(search_title)}' vs '{clean_text(movie_text)}'")
            if clean_text(search_title) in clean_text(movie_text) or clean_text(movie_text) in clean_text(search_title):
                results.append((page, movie_text))
    return results

# CSRF 토큰 가져오기
session = requests.Session()
initial_response = session.get(base_url)
initial_soup = BeautifulSoup(initial_response.text, 'html.parser')
csrf_token = initial_soup.find('input', {'name': 'CSRFToken'})['value']

# 사용자 입력
search_title = input("검색할 영화 제목을 입력하세요: ")

# 병렬 요청
page_range = range(1, 2000)  # 1~200페이지 병렬 요청
all_results = []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(fetch_page, page, csrf_token, search_title) for page in page_range]
    for future in futures:
        result = future.result()
        if result:  # 결과가 존재할 경우만 추가
            all_results.extend(result)

# 최종 결과 출력
if all_results:
    # print("\n찾은 영화 목록:")
    for page, title in all_results:
        if title.strip():  # 타이틀이 공백이 아닐 경우만 출력
            print(f"페이지 {page}: {title}")
else:
    print(f"'{search_title}'에 해당하는 영화를 찾을 수 없습니다.")


KeyboardInterrupt: 